# How do I validate a single field?

## Write `validate_<Field Name>`

If the field name is X, then we can write a method called `validate_X` with the following signature:

    def validate_X(self, json_obj, value_of_x)

In the above signature, `json_obj` is the object passed into the constructor to create the object. In case we need some other information from that structure. `value_of_x` is the value of the field being validated. Here is an example:

In [1]:
import string

import schematics


VALID_CHARS = set(string.ascii_letters + string.digits + "_")


class UserMetadata(schematics.Model):
    name = schematics.types.StringType(required=True)
    display_name = schematics.types.StringType(serialized_name="displayName")
    description = schematics.types.StringType()
    
    def validate_name(self, json_obj, name):
        invalid_chars = "".join(set(name) - VALID_CHARS)
        if invalid_chars:
            raise schematics.exceptions.ValidationError(
                f"Expect only letters, digits and underscore"
                f", but found {invalid_chars!r}"
                f" in {name}"
            )


def verify_it(model_instance):
    try:
        model_instance.validate()
    except schematics.exceptions.DataError as error:
        print(f"Error: {error}")
    else:
        print("Valid")

In [2]:
# This should be valid
verify_it(UserMetadata(dict(name="user_1")))

Valid


In [3]:
# Invalid chars found
verify_it(UserMetadata(dict(name="user-1#")))

Error: {"name": ["Expect only letters, digits and underscore, but found '#-' in user-1#"]}


### What Is the Use of `json_obj`?

Sometimes, we want to validate a field against another field, this is where the `json_obj` comes in. In the following example, we create a `UnixUser` object and if the user is root, then that user must be an admin.

In [4]:
import schematics

class UnixUser(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
    def validate_is_admin(self, json_obj, is_admin_value):
        if json_obj["alias"] == "root" and is_admin_value is not True:
            raise schematics.exceptions.ValidationError(
                "root must be an admin."
            )

In [5]:
# Non-root
verify_it(UnixUser(dict(alias="johan", is_admin=True)))

Valid


In [6]:
# Root, is admin
verify_it(UnixUser(dict(alias="root", is_admin=True)))

Valid


In [7]:
# Invalid
verify_it(UnixUser(dict(alias="root")))

Error: {"is_admin": ["root must be an admin."]}


## Write a Validation Function

If we found having to perform the same validation for different fields, we can write a validation function and use that:

In [8]:
import re

import schematics


def validate_simple_us_phone(value):
    """
    Validate a phone number.
    
    For the sake of simplicity, we only validate NNN-NNN-NNNN.
    """
    valid_pattern = re.compile(r"\d{3}-\d{3}-\d{4}")
    if not valid_pattern.match(value):
        raise schematics.exceptions.ValidationError(f"Invalid phone: {value}")


class Contact(schematics.Model):
    name = schematics.types.StringType(required=True)
    mobile_phone = schematics.types.StringType(validators=[validate_simple_us_phone])
    work_phone = schematics.types.StringType(validators=[validate_simple_us_phone])

In [9]:
# Valid
verify_it(Contact(dict(name="Johan", mobile_phone="425-772-3359")))

Valid


In [10]:
# Bad mobile phone number
verify_it(Contact(dict(name="Johan", mobile_phone="4257723359")))

Error: {"mobile_phone": ["Invalid phone: 4257723359"]}


In [11]:
# Bad phone numbers
verify_it(Contact(dict(name="Johan", mobile_phone="4257723359", work_phone="425-7742")))

Error: {"mobile_phone": ["Invalid phone: 4257723359"], "work_phone": ["Invalid phone: 425-7742"]}


## Create Custom Type

If we found we repeatedly need the same validation, another approach is to write a custom type. In this case, we will write a `SimpleUSPhoneType`. Within this type, we can write a number of `validate_*` functions to validate the value.

In [12]:
import re

import schematics


class SimpleUSPhoneType(schematics.types.StringType):
    """
    For simplicity, we only allow NNN-NNN-NNNN format.
    """
    def validate_first_digit(self, value):
        """The first digit cannot be a 1."""
        if value[0] == "1":
            raise schematics.exceptions.ValidationError(f"First digit must not be 1: {value}")
            
    def validate_format(self, value):
        valid_pattern = re.compile(r"\d{3}-\d{3}-\d{4}")
        if not valid_pattern.match(value):
            raise schematics.exceptions.ValidationError(
                f"Invalid format: {value}"
            )
        


class Contact2(schematics.Model):
    name = schematics.types.StringType(required=True)
    mobile_phone = SimpleUSPhoneType()
    work_phone = SimpleUSPhoneType()

In [13]:
# Valid
verify_it(Contact2(dict(name="Johan", mobile_phone="345-678-9012", work_phone="555-123-4567")))

Valid


In [14]:
# Invalid phone numbers
verify_it(Contact2(dict(name="Johan", mobile_phone="3456789012", work_phone="155-123-4567")))

Error: {"mobile_phone": ["Invalid format: 3456789012"], "work_phone": ["First digit must not be 1: 155-123-4567"]}


In [15]:
# A phone number can be invalid in more than one way
verify_it(Contact2(dict(name="Johan", mobile_phone="12345")))

Error: {"mobile_phone": ["First digit must not be 1: 12345", "Invalid format: 12345"]}


## Validation with Regular Expression

Some field types come with the ability to specify a regular expression to validate. Consider the following example:

In [16]:
class UnixUser(schematics.Model):
    name = schematics.types.StringType(regex=r"[a-z][a-z0-9.-_]+", required=True)

In [17]:
verify_it(UnixUser(dict(name="anna1")))

Valid


In [18]:
verify_it(UnixUser(dict(name="2be-or-not")))

Error: {"name": ["String value did not match validation regex."]}


In [19]:
verify_it(UnixUser(dict(name="MonkeyDo")))

Error: {"name": ["String value did not match validation regex."]}
